# Part one of the variables

In [1]:
import pandas as pd
from functools import reduce #necessary to create the last dataframe
df = pd.read_csv(r"C:\Users\hp pc\Desktop\Tesi\Smart cities\Databases\4. Citizen Satisfaction\Public Perception\urb_percep_1.csv", encoding = "ISO-8859-1",index_col=0)
del df["UNIT"] #Delete the unit of measurement
# Rename cities in English
df = df.replace(["Athina"],"Athens")
df = df.replace(['Bruxelles / Brussel'],'Bruxelles')
df = df.replace(['København'],'Copenhagen')
df = df.replace(['Helsinki / Helsingfors'],'Helsinki')
df = df.replace(['Madrid'],'Madrid')
df = df.replace(['Paris'],'Paris')
df = df.replace(['Praha'],'Prague')
df = df.replace(['Roma'],'Rome')
df = df.replace(['Stockholm'],'Stockholm')
df = df.replace(['Warszawa'],'Warsaw')
df = df.replace(['Wien'],'Vienna')
df = df.replace(['Zagreb'],'Zagreb')

#these are the cities that we analyse in our study
cities=["Athens","Bruxelles","Copenhagen", "Helsinki", "Madrid", "Paris", "Prague", "Rome", 
        "Stockholm", "Warsaw","Vienna", "Zagreb"]

df.sort_values(by=['CITIES'], inplace=True, ascending=True) # sort by cities
df["TIME"]=df.index # Create a column with the date, which is set as index by default
df.reset_index(drop=True, inplace=True)  # reset index
indicators = list(dict.fromkeys(df["INDIC_UR"])) # make list containing names of the variables (no duplicates)

unav_cities=[] # create an empty list that will be filled with cities which don't have available data
dfList=[] #create an empty list that will filled with dataframes
for nome_indicatore in indicators:
    df1=df[df.INDIC_UR == nome_indicatore] 
    df1=df1[df.Value != ':'] #select df where values in certain column are not ":", meaning removing rows with no data
    df1["Value"]=[(float(s.replace(" ",""))) for s in df1["Value"]] # convert Value column into int 
    for city in cities: 
        if df1[df1['CITIES'].str.contains(city)].empty: #if one city in our sample is not in the df:
            unav_cities.append(((city +" has no data for " + nome_indicatore))) # put the name of the city in the list of cities with data NA
    #create a subdataframe for each indicator
        else:
            sub_df=df1[df1.CITIES == city] #take the subdataframe with the name of each city 
            sub_df=sub_df[sub_df.TIME==max(sub_df["TIME"])] # get the row where date is the most recent
            dfList.append(sub_df)
df=(pd.concat(dfList,ignore_index=False)) # This df containes all variables: it needs to be splitted...

final_dfs=[]     #...split the previous df and creating the complete one
for indicator in indicators:
    df1=df[df.INDIC_UR == indicator] 
    df1 = df1.rename(columns={"Value": indicator}) #rename "Value" with the name of the variable 
    del df1["INDIC_UR"] # delete the column filled with the name of the variable. 
    df1.reset_index(drop=True, inplace=True) 
    df1.sort_values("CITIES", axis=0, ascending=True, inplace=True)
    final_dfs.append(df1) # this is the list of df that need to be merged
df= reduce(lambda x, y: pd.merge(x, y, on = 'CITIES'), final_dfs) #merge dfs on CITIES one next to the other
del df["TIME_y"] # (before deleting check if columns are equals)
del df["TIME_x"]
df

C:\Users\hp pc\Documents\Python-Anaconda\lib\site-packages\ipykernel_launcher.py:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,CITIES,The public administration of the city can be trusted: strongly agree,This city is a healthy city to live in: somewhat agree,The public administration of the city can be trusted: somewhat agree,This city is a healthy city to live in: strongly agree,This city spends its resources in a responsible way: somewhat agree,This city spends its resources in a responsible way: strongly agree,"When you contact administrative services of this city, they help you efficiently: somewhat agree",You are satisfied to live in this city: strongly agree,You are satisfied to live in this city: somewhat agree,"When you contact administrative services of this city, they help you efficiently: strongly agree"
0,Athens,6.0,11.6,36.0,4.7,10.7,4.1,37.0,21.0,46.0,5.0
1,Bruxelles,11.0,50.1,58.0,15.9,23.9,3.2,52.0,40.0,47.0,10.0
2,Copenhagen,28.0,48.0,46.0,9.4,39.5,4.0,44.0,76.0,21.0,16.0
3,Helsinki,22.0,57.6,51.0,23.7,47.1,6.9,51.0,63.0,28.0,7.0
4,Madrid,13.0,47.3,25.0,18.5,28.3,6.7,28.0,54.0,33.0,12.0
5,Paris,13.0,47.4,50.0,16.7,36.9,4.0,45.0,41.0,46.0,7.0
6,Prague,4.0,37.3,32.0,7.8,33.9,6.1,43.0,45.0,46.0,4.0
7,Rome,2.0,42.0,24.0,9.7,17.9,7.8,25.0,33.0,47.0,2.0
8,Stockholm,25.0,48.3,52.0,20.4,48.3,13.0,45.0,85.0,12.0,9.0
9,Vienna,20.0,46.0,56.0,44.8,38.4,10.6,56.0,77.0,19.0,11.0


In [2]:
# RENAME COLUMNS USING EASIER NAMES and 
df.columns=["CITIES","adm_tr_str","healthy_city_som","adm_tr_som", 
            "healthy_city_str", "good_spending_som", "good_spending_str", "adm_ser_som", 
            "city_satisf_str", "city_satisf_som", "adm_ser_str",
            ]

# CREATE NEW COLUMNS AS SUM OF VARIABLES "STRONG AGREE" AND "SOMEWHAT AGREE"
df["adm_satisf"]=df["adm_ser_str"] + df["adm_ser_som"] 
df["pub_adm_trust"]=df["adm_tr_str"] + df["adm_tr_som"] 
df["healthy_city"]=df["healthy_city_str"] + df["healthy_city_som"] 
df["good_spending"]=df["good_spending_str"] + df["good_spending_som"] 
df["city_satisf"]=df["city_satisf_str"] + df["city_satisf_som"] 

# DELETE USELESS COLUMNS
del df["adm_ser_str"]
del df["adm_ser_som"] 
del df["adm_tr_str"]
del df["adm_tr_som"] 
del df["healthy_city_str"]
del df["healthy_city_som"] 
del df["good_spending_str"]
del df["good_spending_som"]
del df["city_satisf_str"] 
del df["city_satisf_som"] 

df

,CITIES,adm_satisf,pub_adm_trust,healthy_city,good_spending,city_satisf
0,Athens,42.0,42.0,16.3,14.8,67.0
1,Bruxelles,62.0,69.0,66.0,27.1,87.0
2,Copenhagen,60.0,74.0,57.4,43.5,97.0
3,Helsinki,58.0,73.0,81.3,54.0,91.0
4,Madrid,40.0,38.0,65.8,35.0,87.0
5,Paris,52.0,63.0,64.1,40.9,87.0
6,Prague,47.0,36.0,45.1,40.0,91.0
7,Rome,27.0,26.0,51.7,25.7,80.0
8,Stockholm,54.0,77.0,68.7,61.3,97.0
9,Vienna,67.0,76.0,90.8,49.0,96.0


In [3]:
# Apply min-max normalization formula to each value in df 
df["adm_satisf"]=[round(((s-min(df.adm_satisf))/(max(df.adm_satisf)-min(df.adm_satisf)))*10,1) for s in df["adm_satisf"]]
df["pub_adm_trust"]=[round(((s-min(df.pub_adm_trust))/(max(df.pub_adm_trust)-min(df.pub_adm_trust)))*10,1) for s in df["pub_adm_trust"]]
df["healthy_city"]=[round(((s-min(df.healthy_city))/(max(df.healthy_city)-min(df.healthy_city)))*10,1) for s in df["healthy_city"]]
df["good_spending"]=[round(((s-min(df.good_spending))/(max(df.good_spending)-min(df.good_spending)))*10,1) for s in df["good_spending"]]
df["city_satisf"]=[round(((s-min(df.city_satisf))/(max(df.city_satisf)-min(df.city_satisf)))*10,1) for s in df["city_satisf"]]

# create a column called for the overall satisfaction. Here we compute the average among the two 
df["overall_sat"]=(df["healthy_city"] + df["city_satisf"])/2 
#and delete the two
del df["healthy_city"]
del df["city_satisf"]

part1=df
part1

,CITIES,adm_satisf,pub_adm_trust,good_spending,overall_sat
0,Athens,3.8,3.1,0.0,0.00
1,Bruxelles,8.8,8.4,2.6,6.70
2,Copenhagen,8.2,9.4,6.2,7.75
3,Helsinki,7.8,9.2,8.4,8.35
4,Madrid,3.2,2.4,4.3,6.65
5,Paris,6.2,7.3,5.6,6.55
6,Prague,5.0,2.0,5.4,5.95
7,Rome,0.0,0.0,2.3,4.55
8,Stockholm,6.8,10.0,10.0,8.50
9,Vienna,10.0,9.8,7.4,9.85


# Part two of the variables

In [4]:
import pandas as pd
from functools import reduce #necessary to create the last dataframe
df = pd.read_csv(r"C:\Users\hp pc\Desktop\Tesi\Smart cities\Databases\4. Citizen Satisfaction\Public Perception\urb_percep_2.csv", encoding = "ISO-8859-1",index_col=0)
del df["UNIT"] #Delete the unit of measurement
# Rename cities in English
df = df.replace(["Athina (greater city)"],"Athens")
df = df.replace(['Bruxelles / Brussel'],'Bruxelles')
df = df.replace(['København (greater city)'],'Copenhagen')
df = df.replace(['Helsinki / Helsingfors (greater city)'],'Helsinki')
df = df.replace(['Madrid'],'Madrid')
df = df.replace(['Paris (greater city)'],'Paris')
df = df.replace(['Praha'],'Prague')
df = df.replace(['Roma'],'Rome')
df = df.replace(['Stockholm (greater city)'],'Stockholm')
df = df.replace(['Warszawa'],'Warsaw')
df = df.replace(['Wien'],'Vienna')
df = df.replace(['Zagreb'],'Zagreb')

#these are the cities that we analyse in our study
cities=["Athens","Bruxelles","Copenhagen", "Helsinki", "Madrid", "Paris", "Prague", "Rome", 
        "Stockholm", "Warsaw","Vienna", "Zagreb"]

df.sort_values(by=['CITIES'], inplace=True, ascending=True) # sort by cities
df["TIME"]=df.index # Create a column with the date, which is set as index by default
df.reset_index(drop=True, inplace=True)  # reset index
indicators = list(dict.fromkeys(df["INDIC_UR"])) # make list containing names of the variables (no duplicates)

unav_cities=[] # create an empty list that will be filled with cities which don't have available data
dfList=[] #create an empty list that will filled with dataframes
for nome_indicatore in indicators:
    df1=df[df.INDIC_UR == nome_indicatore] 
    for city in cities: 
        if df1[df1['CITIES'].str.contains(city)].empty: #if one city in our sample is not in the df:
            unav_cities.append(((city +" has no data for " + nome_indicatore))) # put the name of the city in the list of cities with data NA
    #create a subdataframe for each indicator
        else:
            sub_df=df1[df1.CITIES == city] #take the subdataframe with the name of each city 
            sub_df=sub_df[sub_df.TIME==max(sub_df["TIME"])] # get the row where date is the most recent
            dfList.append(sub_df)
df=(pd.concat(dfList,ignore_index=False)) # This df containes all variables: it needs to be splitted...

final_dfs=[]     #...split the previous df and creating the complete one
for indicator in indicators:
    df1=df[df.INDIC_UR == indicator] 
    df1 = df1.rename(columns={"Value": indicator}) #rename "Value" with the name of the variable 
    del df1["INDIC_UR"] # delete the column filled with the name of the variable. 
    df1.reset_index(drop=True, inplace=True) 
    df1.sort_values("CITIES", axis=0, ascending=True, inplace=True)
    final_dfs.append(df1) # this is the list of df that need to be merged
df= reduce(lambda x, y: pd.merge(x, y, on = 'CITIES'), final_dfs) #merge dfs on CITIES one next to the other
del df["TIME_y"] # (before deleting check if columns are equals)
del df["TIME_x"]
df

,CITIES,There is corruption in my local public administration: strongly agree,I am satisfied with the amount of time it takes to get a request solved by my local public administration: strongly agree,I am satisfied with the amount of time it takes to get a request solved by my local public administration: somewhat agree,The procedures used by my local public administration are straightforward and easy to understand: strongly agree,The procedures used by my local public administration are straightforward and easy to understand: somewhat agree,The fees charged by my local public administration are reasonable: strongly agree,The fees charged by my local public administration are reasonable: somewhat agree,There is corruption in my local public administration: somewhat agree
0,Athens,36.0,12.4,25.0,10.0,30.5,9.8,16.6,22.4
1,Bruxelles,13.8,30.3,32.4,42.7,36.1,26.1,35.8,14.6
2,Copenhagen,5.1,25.8,33.2,25.1,32.9,22.3,39.4,11.6
3,Helsinki,13.1,11.9,27.2,16.2,23.6,21.3,39.6,21.0
4,Madrid,25.5,25.9,28.1,25.6,31.8,28.5,22.3,14.3
5,Paris,11.9,21.0,33.4,34.3,25.4,18.3,34.9,18.6
6,Prague,25.5,11.7,29.7,14.7,26.9,16.4,52.5,21.6
7,Rome,37.3,1.3,14.2,3.6,23.5,2.2,24.0,39.9
8,Stockholm,6.2,13.8,28.8,10.0,34.9,26.4,36.1,16.3
9,Vienna,4.9,23.9,43.9,24.7,42.1,25.9,44.7,17.7


In [5]:
# RENAME COLUMNS USING EASIER NAMES and 
df.columns=["CITIES","pub_corr_str","sat_time_str","sat_time_som", 
            "easy_proc_str", "easy_proc_som", "fees_reas_str", "fees_reas_som", 
            "pub_corr_som",
            ]

# CREATE NEW COLUMNS AS SUM OF VARIABLES "STRONG AGREE" AND "SOMEWHAT AGREE"
df["pub_corr"]=df["pub_corr_str"] + df["pub_corr_som"] 
df["sat_time"]=df["sat_time_str"] + df["sat_time_som"] 
df["easy_proc"]=df["easy_proc_str"] + df["easy_proc_som"] 
df["fees_reas"]=df["fees_reas_str"] + df["fees_reas_som"] 

# DELETE USELESS COLUMNS
del df["pub_corr_str"]
del df["pub_corr_som"] 
del df["sat_time_str"]
del df["sat_time_som"] 
del df["easy_proc_str"]
del df["easy_proc_som"] 
del df["fees_reas_str"]
del df["fees_reas_som"] 

part2=df
part2

,CITIES,pub_corr,sat_time,easy_proc,fees_reas
0,Athens,58.4,37.4,40.5,26.4
1,Bruxelles,28.4,62.7,78.8,61.9
2,Copenhagen,16.7,59.0,58.0,61.7
3,Helsinki,34.1,39.1,39.8,60.9
4,Madrid,39.8,54.0,57.4,50.8
5,Paris,30.5,54.4,59.7,53.2
6,Prague,47.1,41.4,41.6,68.9
7,Rome,77.2,15.5,27.1,26.2
8,Stockholm,22.5,42.6,44.9,62.5
9,Vienna,22.6,67.8,66.8,70.6


# MERGE DATAFRAME CREATE AN INDEX FOR PUBLIC PERCEPTION AND CITIZEN SATISFACTION

In [6]:
df = pd.merge(part1, part2, on=['CITIES'])
df

,CITIES,adm_satisf,pub_adm_trust,good_spending,overall_sat,pub_corr,sat_time,easy_proc,fees_reas
0,Athens,3.8,3.1,0.0,0.00,58.4,37.4,40.5,26.4
1,Bruxelles,8.8,8.4,2.6,6.70,28.4,62.7,78.8,61.9
2,Copenhagen,8.2,9.4,6.2,7.75,16.7,59.0,58.0,61.7
3,Helsinki,7.8,9.2,8.4,8.35,34.1,39.1,39.8,60.9
4,Madrid,3.2,2.4,4.3,6.65,39.8,54.0,57.4,50.8
5,Paris,6.2,7.3,5.6,6.55,30.5,54.4,59.7,53.2
6,Prague,5.0,2.0,5.4,5.95,47.1,41.4,41.6,68.9
7,Rome,0.0,0.0,2.3,4.55,77.2,15.5,27.1,26.2
8,Stockholm,6.8,10.0,10.0,8.50,22.5,42.6,44.9,62.5
9,Vienna,10.0,9.8,7.4,9.85,22.6,67.8,66.8,70.6


In [7]:
# Apply min-max normalization formula to each value in df 
df["adm_satisf"]=[round(((s-min(df.adm_satisf))/(max(df.adm_satisf)-min(df.adm_satisf)))*10,1) for s in df["adm_satisf"]]
df["pub_adm_trust"]=[round(((s-min(df.pub_adm_trust))/(max(df.pub_adm_trust)-min(df.pub_adm_trust)))*10,1) for s in df["pub_adm_trust"]]
df["good_spending"]=[round(((s-min(df.good_spending))/(max(df.good_spending)-min(df.good_spending)))*10,1) for s in df["good_spending"]]
df["overall_sat"]=[round(((s-min(df.overall_sat))/(max(df.overall_sat)-min(df.overall_sat)))*10,1) for s in df["overall_sat"]]
df["sat_time"]=[round(((s-min(df.sat_time))/(max(df.sat_time)-min(df.sat_time)))*10,1) for s in df["sat_time"]]
df["easy_proc"]=[round(((s-min(df.easy_proc))/(max(df.easy_proc)-min(df.easy_proc)))*10,1) for s in df["easy_proc"]]
df["fees_reas"]=[round(((s-min(df.fees_reas))/(max(df.fees_reas)-min(df.fees_reas)))*10,1) for s in df["fees_reas"]]

# IN THIS CASE WE APPLY THE MIN-MAX REVERSE FORMULA 
df["pub_corr"]=[round(((max(df.pub_corr)-s)/(max(df.pub_corr)-min(df.pub_corr)))*10,1) for s in df["pub_corr"]]

#CREATE LAST COLUMN
df['Citizen_satisfaction_Index'] = round(df.mean(axis=1),2)
df

,CITIES,adm_satisf,pub_adm_trust,good_spending,overall_sat,pub_corr,sat_time,easy_proc,fees_reas,Citizen_satisfaction_Index
0,Athens,3.8,3.1,0.0,0.0,3.1,4.2,2.6,0.0,2.10
1,Bruxelles,8.8,8.4,2.6,6.8,8.1,9.0,10.0,8.0,7.71
2,Copenhagen,8.2,9.4,6.2,7.9,10.0,8.3,6.0,8.0,8.00
3,Helsinki,7.8,9.2,8.4,8.5,7.1,4.5,2.5,7.8,6.98
4,Madrid,3.2,2.4,4.3,6.8,6.2,7.4,5.9,5.5,5.21
5,Paris,6.2,7.3,5.6,6.6,7.7,7.4,6.3,6.1,6.65
6,Prague,5.0,2.0,5.4,6.0,5.0,5.0,2.8,9.6,5.10
7,Rome,0.0,0.0,2.3,4.6,0.0,0.0,0.0,0.0,0.86
8,Stockholm,6.8,10.0,10.0,8.6,9.0,5.2,3.4,8.2,7.65
9,Vienna,10.0,9.8,7.4,10.0,9.0,10.0,7.7,10.0,9.24


In [8]:
Citizen_satisfaction_Index = df[["CITIES", "Citizen_satisfaction_Index"]]
Citizen_satisfaction_Index

,CITIES,Citizen_satisfaction_Index
0,Athens,2.10
1,Bruxelles,7.71
2,Copenhagen,8.00
3,Helsinki,6.98
4,Madrid,5.21
5,Paris,6.65
6,Prague,5.10
7,Rome,0.86
8,Stockholm,7.65
9,Vienna,9.24


In [9]:
%store Citizen_satisfaction_Index

Stored 'Citizen_satisfaction_Index' (DataFrame)
